# Package/Library Importing

In [ ]:
!pip install haversine
import networkx as nx
import pandas as pd
import time
import signal
from haversine import haversine
import psutil
import matplotlib.pyplot as plt
import heapq

# User Input functions

In [ ]:
# Exit commands list
exits = ["Quit", "quit", "QUIT", "Exit", "exit", "EXIT"]

# User input and verification
def get_check_input():
  city_in = input()
  city = input_check(city_in)
  return city

def input_check(city): # modded with minimal assist from ChatGPT
  if city in exits:
    return "Exit"
  else:
    while city not in city_coor_list['city_name'].values:
      print("Invalid city.\n")
      city = input("Enter city name: ")
      if city in exits: # This replaced a recursion call
        return "Exit"
    return city


# Importing City Data from file

In [ ]:
city_coor_list = pd.read_csv('coordinates.csv')
city_coor_list['city_name'] = city_coor_list['city_name'].str.replace("_", " ")
print(city_coor_list.shape)
city_coor_list.head()

(46, 3)


,city_name,latitude,longitude
0,Abilene,38.922028,-97.266667
1,Andover,37.686840,-97.165775
2,Anthony,37.157517,-98.072895
3,Argonia,37.267017,-97.772681
4,Attica,37.242127,-98.235197


# Build the Initial Graph

In [ ]:
# Initialize the graph
graph = nx.Graph()

# Add vertices to the graph
for index, row in city_coor_list.iterrows():
    graph.add_node(row['city_name'], latitude=row['latitude'], longitude=row['longitude'])

# Load adjacency pairs from text file and add edges
with open('Adjacencies.txt', 'r') as file:
    adjacency_data = file.readlines()

for line in adjacency_data:
    cities = line.strip().split()
    if len(cities) == 2:
        city1, city2 = cities
        graph.add_edge(city1.replace("_", " "), city2.replace("_", " "))

# Convert to a bidirectional graph
G = nx.DiGraph(graph)

In [ ]:
# Print the graph to verify vertices and their connections
print("\nGraph Structure:")
print(graph.nodes(data=True))
print(graph.edges())


Graph Structure:
[('Abilene', {'latitude': 38.9220277, 'longitude': -97.2666667}), ('Andover', {'latitude': 37.6868403, 'longitude': -97.1657752}), ('Anthony', {'latitude': 37.1575168, 'longitude': -98.0728946}), ('Argonia', {'latitude': 37.2670166, 'longitude': -97.7726807}), ('Attica', {'latitude': 37.2421271, 'longitude': -98.2351967}), ('Augusta', {'latitude': 37.6913277, 'longitude': -97.0537108}), ('Bluff City', {'latitude': 37.0760844, 'longitude': -97.8793212}), ('Caldwell', {'latitude': 37.0346731, 'longitude': -97.6179436}), ('Cheney', {'latitude': 37.632882, 'longitude': -97.789442}), ('Clearwater', {'latitude': 37.5166968, 'longitude': -97.5325458}), ('Coldwater', {'latitude': 37.2574937, 'longitude': -99.3549149}), ('Derby', {'latitude': 37.5517122, 'longitude': -97.2867892}), ('El Dorado', {'latitude': 37.8098997, 'longitude': -96.8943313}), ('Emporia', {'latitude': 38.3792991, 'longitude': -96.2615044}), ('Florence', {'latitude': 38.2434672, 'longitude': -96.9378672}), 

# Sample function to perform test search

In [ ]:
# Note: This entire cell is mostly from ChatGPT.
# This is a sample run of the graph/monitoring functionality.
# Modifications were made by myself.

# Function to handle timeouts
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException()

# Setting the timeout
signal.signal(signal.SIGALRM, timeout_handler)
timeout_duration = 5  # 5000 milliseconds
signal.setitimer(signal.ITIMER_REAL, timeout_duration)

try:
    # Start timing the search
    start_time = time.time()

    # Perform BFS
    bfs_edges = list(nx.bfs_edges(graph, source='Andover'))

    # Calculate elapsed time
    elapsed_time = time.time() - start_time

    # Track memory usage
    memory_usage = psutil.Process().memory_info().rss / (1024 * 1024)  # in MB

    # Distance calculation (example for edges in BFS path)
    total_distance = 0
    for u, v in bfs_edges:
        u_coords = (graph.nodes[u]['latitude'], graph.nodes[u]['longitude'])
        v_coords = (graph.nodes[v]['latitude'], graph.nodes[v]['longitude'])
        total_distance += haversine(u_coords, v_coords)

    # Print search results
    print(f"BFS Traversal from Andover: {bfs_edges}")
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")
    print(f"Memory Usage: {memory_usage:.2f} MB")
    print(f"Total Distance Traveled: {total_distance:.2f} km")

except TimeoutException:
    print("The search exceeded the allowed time and was stopped.")

BFS Traversal from Andover: [('Andover', 'Winfield'), ('Andover', 'Leon'), ('Andover', 'Towanda'), ('Andover', 'Augusta'), ('Andover', 'Mulvane'), ('Andover', 'Newton'), ('Leon', 'Wichita'), ('Towanda', 'El Dorado'), ('Augusta', 'Emporia'), ('Mulvane', 'Mayfield'), ('Mulvane', 'South Haven'), ('Mulvane', 'Cheney'), ('Newton', 'Hutchinson'), ('Newton', 'Haven'), ('Newton', 'McPherson'), ('Wichita', 'Derby'), ('El Dorado', 'Hillsboro'), ('Mayfield', 'Bluff City'), ('Mayfield', 'Wellington'), ('Mayfield', 'Oxford'), ('South Haven', 'Caldwell'), ('Cheney', 'Kingman'), ('Cheney', 'Pratt'), ('Cheney', 'Clearwater'), ('Hutchinson', 'Florence'), ('McPherson', 'Marion'), ('McPherson', 'Salina'), ('Hillsboro', 'Lyons'), ('Bluff City', 'Anthony'), ('Bluff City', 'Kiowa'), ('Caldwell', 'Argonia'), ('Pratt', 'Coldwater'), ('Pratt', 'Sawyer'), ('Pratt', 'Zenda'), ('Marion', 'Abilene'), ('Marion', 'Manhattan'), ('Anthony', 'Harper'), ('Kiowa', 'Attica'), ('Argonia', 'Rago'), ('Coldwater', 'Greensburg

In [ ]:
# subax1 = plt.subplot(121)
# nx.draw(G, with_labels=True, font_weight='bold')
# subax2 = plt.subplot(122)
# nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
def directions():
  print("Enter the name of the starting city:\n" + "\ttype Quit to exit:\n")
  city_1 = get_check_input()
  if city_1 == "Exit":
    print("Goodbye")
    farewell = True
  else:
    print("You're starting at the city of " + city_1)

  if farewell == False:
    print("Enter the name of the ending city:\n" + "\ttype Quit to exit:\n")
    city_2 = get_check_input()
    if city_2 == "Exit":
      print("Goodbye")
      farewell = True
    else: print("You're ending at the city of " + city_2)
  else: city_2 = "Exit"
  return city_1, city_2

In [ ]:
def search_method():
  print("What search method would you like to use?\n\t1)Breadth-First\n\t2)Depth-First\n\t3)Iterative Deepening DFS\n\t4)Best-First\n\t5)A*star")
  print("6) Choose New Cities")
  print("Type Quit to exit")
  choice = input()
  if choice in exits:
    print("Goodbye")
    farewell = True
  elif choice == "1":
    print("Breadth-First Search")
  elif choice == "2":
    print("Depth-First Search")
  elif choice == "3":
    print("Iterative Deepening DFS")
  elif choice == "4":
    print("Best-First Search")
  elif choice == "5":
    print("A*star")
  elif choice == "6"
    print("Choose New Cities")
  else:
    print("Invalid choice\nPlease enter a valid choice")
    choice = input()
  return choice

Enter the name of the starting city:
	type Quit to exit:



KeyboardInterrupt: Interrupted by user